This homework is made by:
* Masoumeh Bakhtiariziabari (11813105)
* Marianne de Heer Kloots (11138351)
* Tharangni Sivaji (11611065)

# Theoretical Part [15 pts]

## 1. Hypothesis Testing – The problem of multiple comparisons [5 points]
Experimentation in AI often happens like this: 
1. Modify/Build an algorithm
2. Compare the algorithm to a baseline by running a hypothesis test.
3. If not significant, go back to step A
4. If significant, start writing a paper. 

How many hypothesis tests, m, does it take to get to (with Type I error for each test = α):
* P(m<sup>th</sup> experiment gives significant result | m experiments lacking power to reject H<sub>0</sub>)?
* P(at least one significant result | m experiments lacking power to reject H<sub>0</sub>)?

<div style="background-color: lightyellow">
*answer here*
</div>


## 2. Bias and unfairness in Interleaving experiments [10 points]
Balance interleaving has been shown to be biased in a number of corner cases. An example was given during the lecture with two ranked lists of length 3 being interleaved, and a randomly clicking population of users that resulted in algorithm A winning ⅔ of the time, even though in theory the percentage of wins should be 50% for both algorithms. Can you come up with a situation of two ranked lists of length 3 and a distribution of clicks over them for which Team-draft interleaving is unfair to the better algorithm?

<div style="background-color: lightyellow">
*answer here*
</div>

# Experimental Part [85 pts]
Commercial search engines use both offline and online approach in evaluating a new search algorithm: they first use an offline test collection to compare the production algorithm (P) with the new experimental algorithm (E); if *E* statistically significantly outperforms *P* with respect to the evaluation measure of their interest, the two algorithms are then compared online through an interleaving experiment.

For the purpose of this homework we will assume that the evaluation measures of interest are:
1. Binary evaluation measures
    1. Precision at rank k,
    2. Recall at rank k,
    3. Average Precision,
2. Multi-graded evaluation measures
    1. Normalized Discounted Cumulative Gain at rank k (nDCG@k),
    2. Expected Reciprocal Rank (ERR).

Further, for the purpose of this homework we will assume that the interleaving algorithms of interest are:
Team-Draft Interleaving (Joachims. "Evaluating retrieval performance using clickthrough data". Text Mining 2003.),
Probabilistic Interleaving (Hofmann, Whiteson, and de Rijke. "A probabilistic method for inferring preferences from clicks." CIKM 2011.).
 
In an interleaving experiment the ranked results of *P* and *E* (against a user query) are interleaved in a single ranked list which is presented to a user. The user then clicks on the results and the algorithm that receives most of the clicks wins the comparison. The experiment is repeated for a number of times (impressions) and the total wins for *P* and *E* are computed. 

A Sign/Binomial Test is then run to examine whether the difference in wins between the two algorithms is statistically significant (or due to chance). Alternatively one can calculate the proportion of times the *E* wins and test whether this proportion, *p*, is greater than *p<sub>0</sub>=*0.5. This is called an 1-sample 1-sided proportion test.

One of the key questions however is **whether offline evaluation and online evaluation outcomes agree with each other**. In this homework you will determine the degree of agreement between offline evaluation measures and interleaving outcomes, by the means of simulations. A similar analysis using actual online data can be found at Chapelle et al. “Large-Scale Validation and Analysis of Interleaved Search Evaluation”.

## <font color='purple'>[Based on Lecture 1]</font>
### Step 1: <font color='darkred'>Simulate Rankings of Relevance for *E* and *P* *(5 points)*</font>

In the first step you will generate pairs of rankings of relevance, for the production *P* and experimental *E*, respectively, for a hypothetical query **q**. Assume a 3-graded relevance, i.e. `{N, R, HR}`. Construct all possible *P* and *E* ranking pairs of length 5. This step should give you about.

Example:

    P: {N N N N N}
    E: {N N N N R}
    …
    P: {HR HR HR HR R}
    E: {HR HR HR HR HR}

(Note 1: If you do not have enough computational power, sample 5000 pair uniformly at random to show your work.)

In [6]:
# TODO: add comments
from itertools import product
from pprint import pprint
import random

algorithms = ['P', 'E']
relevance_grades = {'N', 'HR', 'R'}
rankings = [ranking for ranking in product(relevance_grades, repeat=5)]
algorithm_rankings = [product(alg, rankings) for alg in algorithms]

ranking_pairs = [pair for pair in product(*algorithm_rankings)]
print(len(ranking_pairs))
#pprint(ranking_pairs)

59049


<div style="background-color: lightyellow">
explanation/analysis?
</div>

### Step 2: <font color='darkred'>Implement Evaluation Measures *(10 points)*</font>
Implement 1 binary and 2 multi-graded evaluation measures out of the 7 measures mentioned above. 

(Note 2: Some of the aforementioned measures require the total number of relevant and highly relevant documents in the entire collection – pay extra attention on how to find this)

In [7]:
# code
#p@K function

#import numpy as np
def precision(k,_ranking_pairs):
    
    def calc_precision(i,x):
        rel_counter = 0.0
        prec = 0.0
        for j in range(k):
            if _ranking_pairs[i][x][1][j] == 'HR' or _ranking_pairs[i][x][1][j] == 'R':
                rel_counter += 1
                prec += rel_counter/(1.0+j)
        return prec
    
    prec_list = []
    for i in range(len(_ranking_pairs)):
        p_prec = calc_precision(i,0)
        e_prec = calc_precision(i,1)
        prec_list.append((p_prec/k,e_prec/k))
        #print(p_prec,e_prec)
    
    return prec_list


In [8]:
prec = precision(3,ranking_pairs)
pprint(prec[:100])

[(1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 0.6666666666666666),
 (1.0, 0.6666666666666666),
 (1.0, 0.6666666666666666),
 (1.0, 0.6666666666666666),
 (1.0, 0.6666666666666666),
 (1.0, 0.6666666666666666),
 (1.0, 0.6666666666666666),
 (1.0, 0.6666666666666666),
 (1.0, 0.6666666666666666),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (1.0, 0.5555555555555555),
 (1.0, 0.5555555555555555),
 (1.0, 0.5555555555555555),
 (1.0, 0.5555555555555555),
 (1.0, 0.5555555555555555),
 (1.0, 0.5555555555555555),
 (1.0, 0.5555555555555555),
 (1.0, 0.5555555555555555),
 (1.0, 0.5555555555555555),
 (1.0, 0.3333333333333333),
 (1.0, 0.3333333333333333),
 (1.0, 0.3333333333333333),
 (1.0, 0.3333333333333333),
 (1.0, 0.3333333333333333),
 (1.0, 0.3333333333333333),
 (1.0, 0.3333333333333333),
 (1.0, 0.3333333333333333),
 (1.0, 0.3333333333333333),
 (1.0, 0.5

In [9]:
import numpy as np

def average_prec(k_max,_ranking_pairs):
    temp_list = []
    ap_list = [[0,0] for i in range(len(_ranking_pairs))]
    for k in range(1,k_max+1):
        temp_list = precision(k,_ranking_pairs)
        #print(k,temp_list)
        for index,item in enumerate(temp_list):
            ap_list[index] = (np.array(ap_list[index])+np.array(item)).tolist()
    #print(ap_list)
    for item in ap_list:
        item[:] = [i / k_max for i in item]
    return ap_list       
    

In [10]:
k_max = 5
avg_prec = average_prec(k_max,ranking_pairs)
for i in range(100):#(len(ranking_pairs)):
    print(ranking_pairs[i][0], '%.3f'%avg_prec[i][0],'\t',ranking_pairs[i][1],'\t','%.3f'%avg_prec[i][1])

('P', ('R', 'R', 'R', 'R', 'R')) 1.000 	 ('E', ('R', 'R', 'R', 'R', 'R')) 	 1.000
('P', ('R', 'R', 'R', 'R', 'R')) 1.000 	 ('E', ('R', 'R', 'R', 'R', 'N')) 	 0.960
('P', ('R', 'R', 'R', 'R', 'R')) 1.000 	 ('E', ('R', 'R', 'R', 'R', 'HR')) 	 1.000
('P', ('R', 'R', 'R', 'R', 'R')) 1.000 	 ('E', ('R', 'R', 'R', 'N', 'R')) 	 0.902
('P', ('R', 'R', 'R', 'R', 'R')) 1.000 	 ('E', ('R', 'R', 'R', 'N', 'N')) 	 0.870
('P', ('R', 'R', 'R', 'R', 'R')) 1.000 	 ('E', ('R', 'R', 'R', 'N', 'HR')) 	 0.902
('P', ('R', 'R', 'R', 'R', 'R')) 1.000 	 ('E', ('R', 'R', 'R', 'HR', 'R')) 	 1.000
('P', ('R', 'R', 'R', 'R', 'R')) 1.000 	 ('E', ('R', 'R', 'R', 'HR', 'N')) 	 0.960
('P', ('R', 'R', 'R', 'R', 'R')) 1.000 	 ('E', ('R', 'R', 'R', 'HR', 'HR')) 	 1.000
('P', ('R', 'R', 'R', 'R', 'R')) 1.000 	 ('E', ('R', 'R', 'N', 'R', 'R')) 	 0.813
('P', ('R', 'R', 'R', 'R', 'R')) 1.000 	 ('E', ('R', 'R', 'N', 'R', 'N')) 	 0.781
('P', ('R', 'R', 'R', 'R', 'R')) 1.000 	 ('E', ('R', 'R', 'N', 'R', 'HR')) 	 0.813
('P', ('R

<div style="background-color: lightyellow">
explanation/analysis?
</div>

In [14]:
#-------------calc ideal dcg based on ground truth [10 HR, 10 R, 10 N]
def calc_ideal_dgc(k):
    idcg_rel = [2]*10 + [1]*10
    idcg = 0
    for index, item in enumerate(idcg_rel):
            idcg += ((2**item) - 1)/(math.log2(2+index))
    return idcg
        

In [25]:
# I used the second formula of DCG from slide 6 of http://www.cs.cornell.edu/courses/cs4300/2013fa/lectures/evaluation-1-4pp.pdf
#nDCG is a way to calculate this measure across many independent ____queries____(http://curtis.ml.cmu.edu/w/courses/index.php/Normalized_discounted_cumulative_gain)
#Normalize DCG at rank n by the DCG value at rank n of the ideal ranking(stanford slide)

import math
def ndcg(max_k,query_index,method):    
    all_dcg_k = []
    
    #--------------------make dcg list for all k-----------------------------
    for r in range(0,max_k):
        if ranking_pairs[query_index][method][1][r] == 'HR':
            rel_num = 2
        elif ranking_pairs[query_index][method][1][r] == 'R':
            rel_num = 1
        else:
            rel_num = 0
            
        if len(all_dcg_k) > 0:   
            all_dcg_k.append((((2**rel_num) - 1)/(math.log2(2+r))) + all_dcg_k[-1])
        else:
            all_dcg_k.append(((2**rel_num) - 1)/(math.log2(2+r)))
            
            
    #----------------calculate ideal dcg------------------------------------      
    idcg = calc_ideal_dgc(max_k)
        
    #--------------------------convert dcg to ndcg--------------------------   
    if(idcg == 0):
        all_dcg_k[:] = [0 for x in all_dcg_k]
    else:
        all_dcg_k[:] = [x / idcg for x in all_dcg_k]
     
    return all_dcg_k

#-----------------------call ndcg function for all 59000 queries------------------
ndcg_list = []
max_k = 5
for i in range(len(ranking_pairs)):
    p_dcg = ndcg(max_k,i,0)
    e_dcg = ndcg(max_k,i,1)
    ndcg_list.append((p_dcg[-1],e_dcg[-1]))

    
    

In [27]:
ndcg_list[:10]

[(0.1828231137664974, 0.1828231137664974),
 (0.1828231137664974, 0.15883579295371644),
 (0.1828231137664974, 0.23079775539205935),
 (0.1828231137664974, 0.15611844322551977),
 (0.1828231137664974, 0.13213112241273878),
 (0.1828231137664974, 0.2040930848510817),
 (0.1828231137664974, 0.23623245484845276),
 (0.1828231137664974, 0.2122451340356718),
 (0.1828231137664974, 0.2842070964740147),
 (0.1828231137664974, 0.15181995136959395)]

In [29]:
counter = 0
for index,item in enumerate(ndcg_list):
    if(counter<100):
        print("query %d"%index,":")
        print(ranking_pairs[index][0], item[0])
        print(ranking_pairs[index][1], item[1],'\n')
    counter += 1
    

query 0 :
('P', ('R', 'R', 'R', 'R', 'R')) 0.1828231137664974
('E', ('R', 'R', 'R', 'R', 'R')) 0.1828231137664974 

query 1 :
('P', ('R', 'R', 'R', 'R', 'R')) 0.1828231137664974
('E', ('R', 'R', 'R', 'R', 'N')) 0.15883579295371644 

query 2 :
('P', ('R', 'R', 'R', 'R', 'R')) 0.1828231137664974
('E', ('R', 'R', 'R', 'R', 'HR')) 0.23079775539205935 

query 3 :
('P', ('R', 'R', 'R', 'R', 'R')) 0.1828231137664974
('E', ('R', 'R', 'R', 'N', 'R')) 0.15611844322551977 

query 4 :
('P', ('R', 'R', 'R', 'R', 'R')) 0.1828231137664974
('E', ('R', 'R', 'R', 'N', 'N')) 0.13213112241273878 

query 5 :
('P', ('R', 'R', 'R', 'R', 'R')) 0.1828231137664974
('E', ('R', 'R', 'R', 'N', 'HR')) 0.2040930848510817 

query 6 :
('P', ('R', 'R', 'R', 'R', 'R')) 0.1828231137664974
('E', ('R', 'R', 'R', 'HR', 'R')) 0.23623245484845276 

query 7 :
('P', ('R', 'R', 'R', 'R', 'R')) 0.1828231137664974
('E', ('R', 'R', 'R', 'HR', 'N')) 0.2122451340356718 

query 8 :
('P', ('R', 'R', 'R', 'R', 'R')) 0.1828231137664974
(

### Step 3: <font color='darkred'>Calculate the 𝛥measure *(0 points)*</font>
For the three measures and all *P* and *E* ranking pairs constructed above calculate the difference: 𝛥measure = measure<sub>E</sub>-measure<sub>P</sub>. Consider only those pairs for which *E* outperforms *P*.


In [24]:
#for ap:
#for i in range(len(ranking_pairs)):
counter = 0
i = 0
while i < len(ranking_pairs) and counter < 100:
    i += 1
    if(avg_prec[i][1]-avg_prec[i][0] > 0):
        counter += 1
        print(ranking_pairs[i][0],'\t',ranking_pairs[i][1],'\t','%.3f'%(avg_prec[i][1]-avg_prec[i][0]))

('P', ('R', 'R', 'R', 'R', 'N')) 	 ('E', ('R', 'R', 'R', 'R', 'R')) 	 0.040
('P', ('R', 'R', 'R', 'R', 'N')) 	 ('E', ('R', 'R', 'R', 'R', 'HR')) 	 0.040
('P', ('R', 'R', 'R', 'R', 'N')) 	 ('E', ('R', 'R', 'R', 'HR', 'R')) 	 0.040
('P', ('R', 'R', 'R', 'R', 'N')) 	 ('E', ('R', 'R', 'R', 'HR', 'HR')) 	 0.040
('P', ('R', 'R', 'R', 'R', 'N')) 	 ('E', ('R', 'R', 'HR', 'R', 'R')) 	 0.040
('P', ('R', 'R', 'R', 'R', 'N')) 	 ('E', ('R', 'R', 'HR', 'R', 'HR')) 	 0.040
('P', ('R', 'R', 'R', 'R', 'N')) 	 ('E', ('R', 'R', 'HR', 'HR', 'R')) 	 0.040
('P', ('R', 'R', 'R', 'R', 'N')) 	 ('E', ('R', 'R', 'HR', 'HR', 'HR')) 	 0.040
('P', ('R', 'R', 'R', 'R', 'N')) 	 ('E', ('R', 'HR', 'R', 'R', 'R')) 	 0.040
('P', ('R', 'R', 'R', 'R', 'N')) 	 ('E', ('R', 'HR', 'R', 'R', 'HR')) 	 0.040
('P', ('R', 'R', 'R', 'R', 'N')) 	 ('E', ('R', 'HR', 'R', 'HR', 'R')) 	 0.040
('P', ('R', 'R', 'R', 'R', 'N')) 	 ('E', ('R', 'HR', 'R', 'HR', 'HR')) 	 0.040
('P', ('R', 'R', 'R', 'R', 'N')) 	 ('E', ('R', 'HR', 'HR', 'R', 'R')

In [30]:
#for nDCG:
#for i in range(len(ranking_pairs)):
counter = 0
i = 0
while i < len(ranking_pairs) and counter < 100:
    i += 1
    if(ndcg_list[i][1]-ndcg_list[i][0] > 0):
        counter += 1
        print(ranking_pairs[i][0],'\t',ranking_pairs[i][1],'\t','%.3f'%(ndcg_list[i][1]-ndcg_list[i][0]))

('P', ('R', 'R', 'R', 'R', 'R')) 	 ('E', ('R', 'R', 'R', 'R', 'HR')) 	 0.048
('P', ('R', 'R', 'R', 'R', 'R')) 	 ('E', ('R', 'R', 'R', 'N', 'HR')) 	 0.021
('P', ('R', 'R', 'R', 'R', 'R')) 	 ('E', ('R', 'R', 'R', 'HR', 'R')) 	 0.053
('P', ('R', 'R', 'R', 'R', 'R')) 	 ('E', ('R', 'R', 'R', 'HR', 'N')) 	 0.029
('P', ('R', 'R', 'R', 'R', 'R')) 	 ('E', ('R', 'R', 'R', 'HR', 'HR')) 	 0.101
('P', ('R', 'R', 'R', 'R', 'R')) 	 ('E', ('R', 'R', 'N', 'R', 'HR')) 	 0.017
('P', ('R', 'R', 'R', 'R', 'R')) 	 ('E', ('R', 'R', 'N', 'HR', 'R')) 	 0.022
('P', ('R', 'R', 'R', 'R', 'R')) 	 ('E', ('R', 'R', 'N', 'HR', 'HR')) 	 0.070
('P', ('R', 'R', 'R', 'R', 'R')) 	 ('E', ('R', 'R', 'HR', 'R', 'R')) 	 0.062
('P', ('R', 'R', 'R', 'R', 'R')) 	 ('E', ('R', 'R', 'HR', 'R', 'N')) 	 0.038
('P', ('R', 'R', 'R', 'R', 'R')) 	 ('E', ('R', 'R', 'HR', 'R', 'HR')) 	 0.110
('P', ('R', 'R', 'R', 'R', 'R')) 	 ('E', ('R', 'R', 'HR', 'N', 'R')) 	 0.035
('P', ('R', 'R', 'R', 'R', 'R')) 	 ('E', ('R', 'R', 'HR', 'N', 'N')) 	 0.

<div style="background-color: lightyellow">
explanation/analysis?
</div>

## <font color='purple'>[Based on Lecture 2]</font>
### Step 4: <font color='darkred'>Implement Interleaving *(15 points)*</font>
Implement 2 interleaving algorithms: (1) Team-Draft Interleaving OR Balanced Interleaving, AND (2), Probabilistic Interleaving. The interleaving algorithms should (a) given two rankings of relevance interleave them into a single ranking, and (b) given the users clicks on the interleaved ranking assign credit to the algorithms that produced the rankings.

(Note 4: Note here that as opposed to a normal interleaving experiment where rankings consists of urls or docids, in our case the rankings consist of relevance labels. Hence in this case (a) you will assume that E and P return different documents, (b) the interleaved ranking will also be a ranking of labels.)


In [33]:
random.choice([True, False])

True

In [38]:
#Team-Draft Interleaving
#P and E as two interleaving lists
for i in range(59000,59001):#(len(ranking_pairs)):
    A = ranking_pairs[i][0][1]
    B = ranking_pairs[i][1][1]
    
    
    team_A = []
    team_B = []
    I = []
    while len(team_A) < len(A) or len(team_B) < len(B):
        RandBit = random.choice([True, False])
        #pick from A
        if len(team_A) < len(team_B) or (len(team_A) == len(team_B) and RandBit == True):
            print("A")
            I.append(A[len(team_A)])
            team_A.append(I[-1])        
        else:
            #pick from B
            print("B")
            I.append(B[len(team_B)])
            team_B.append(I[-1])
            
    print(ranking_pairs[i],'\n')
    print(I,'\n')
    



A
B
A
B
A
B
B
A
B
A
(('P', ('HR', 'HR', 'HR', 'HR', 'HR')), ('E', ('HR', 'N', 'R', 'N', 'HR'))) 

['HR', 'HR', 'HR', 'N', 'HR', 'R', 'N', 'HR', 'HR', 'HR'] 



<div style="background-color: lightyellow">
explanation/analysis?
</div>

## <font color='purple'>[Based on Lecture 3]</font>
### Step 5: <font color='darkred'>Implement User Clicks Simulation *(15 points)*</font>
Having interleaved all the ranking pairs an online experiment could be ran. However, given that we do not have any users (and the entire homework is a big simulation) we will simulate user clicks.

We have considered a number of click models including:
1. Random Click Model (RCM)
2. Position-Based Model (PBM)
3. Simple Dependent Click Model (SDCM)
4. Simple Dynamic Bayesian Network (SDBN)

Consider two different click models, (a) the Random Click Model (RCM), and (b) one out of the remaining 3 aforementioned models. The parameters of some of these models can be estimated using the Maximum Likelihood Estimation (MLE) method, while others require using the Expectation-Maximization (EM) method. Implement the two models so that (a) there is a method that learns the parameters of the model given a set of training data, (b) there is a method that predicts the click probability given a ranked list of relevance labels, (c) there is a method that decides - stochastically - whether a document is clicked based on these probabilities.

Having implemented the two click models, estimate the model parameters using the Yandex Click Log [[file]](https://drive.google.com/file/d/1tqMptjHvAisN1CJ35oCEZ9_lb0cEJwV0/view).

(Note 6: Do not learn the attractiveness parameter *a*<sub>uq</sub>)

In [ ]:
# code

<div style="background-color: lightyellow">
explanation/analysis?
</div>

### Step 6: <font color='darkred'>Simulate Interleaving Experiment *(10 points)*</font>
Having implemented the click models, it is time to run the simulated experiment.

For each of interleaved ranking run N simulations for each one of the click models implemented and measure the proportion *p* of wins for E.
(Note 7: Some of the models above include an attractiveness parameter *a*<sub>uq</sub>. Use the relevance label to assign this parameter by setting *a*<sub>uq</sub> for a document u in the ranked list accordingly. (See [Click Models for Web Search](http://clickmodels.weebly.com/uploads/5/2/2/5/52257029/mc2015-clickmodels.pdf))


In [ ]:
# code

<div style="background-color: lightyellow">
explanation/analysis?
</div>

### Step 7: <font color='darkred'>Results and Analysis *(30 points)*</font>
Compare the results of the offline experiments (i.e. the values of the 𝛥measure) with the results of the online experiment (i.e. proportion of wins), analyze them and reach your conclusions regarding their agreement.
* Use easy to read and comprehend visuals to demonstrate the results;
* Analyze the results on the basis of
    * the evaluation measure used,
    * the interleaving method used,
    * the click model used.
* Report and ground your conclusions.

(Note 8: This is the place where you need to demonstrate your deeper understanding of what you have implemented so far; hence the large number of points assigned. Make sure you clearly do that so that the examiner of your work can grade it accordingly.)

<u>Yandex Click Log File</u>:

The dataset includes user sessions extracted from Yandex logs, with queries, URL rankings and clicks. To allay privacy concerns the user data is fully anonymized. So, only meaningless numeric IDs of queries, sessions, and URLs are released. The queries are grouped only by sessions and no user IDs are provided. The dataset consists of several parts. Logs represent a set of rows, where each row represents one of the possible user actions: query or click.

In the case of a Query:

    SessionID TimePassed TypeOfAction QueryID RegionID ListOfURLs


In the case of a Click:

    SessionID TimePassed TypeOfAction URLID


* `SessionID` - the unique identifier of the user session.
* `TimePassed` - the time elapsed since the beginning of the current session in standard time units.
* `TypeOfAction` - type of user action. This may be either a query (Q), or a click (C).
* `QueryID` - the unique identifier of the request.
* `RegionID` - the unique identifier of the country from which a given query. This identifier may take four values.
* `URLID` - the unique identifier of the document.
* `ListOfURLs` - the list of documents from left to right as they have been shown to users on the page extradition Yandex (top to bottom).


In [ ]:
# code

<div style="background-color: lightyellow">
explanation/analysis?
</div>